### 7章　線形回帰

### 環境準備

#### ライブラリ導入

In [ ]:
# ライブラリ導入
!pip install japanize-matplotlib -q

#### ライブラリインポート

In [ ]:
# ライブラリインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
from IPython.display import display
from sklearn.datasets import fetch_openml
import warnings

#### 環境設定

In [ ]:
# 環境設定
np.set_printoptions(formatter={'float': '{:0.3f}'.format})
pd.options.display.float_format = '{:.3f}'.format
warnings.filterwarnings('ignore')

### データ読み込み

#### 読み込み関数定義

In [ ]:
# 読み込み関数定義

def load_california_housing():
    """California Housing データセットを取得し DataFrame を返す"""
    try:
        data = fetch_openml(name="california", version=2, as_frame=True)
        features = data.data
        target = data.target.astype(float)
        df = pd.concat([features, target.rename("MedianHouseValue")], axis=1)

        print("データの読み込みが完了しました。")
        print(f"サンプル数: {df.shape[0]:,} 件, 特徴量数: {df.shape[1]-1} 個")
        return df
    except Exception as e:
        print("データ読み込みエラー:", e)
        return None

#### 読み込み

In [ ]:
# 読み込み
df = load_california_housing()

#### 内容確認

In [ ]:
# 内容確認

# 先頭5行表示
display(df.head())

### データ加工

#### 特徴量MedIncの抽出

In [ ]:
# 特徴量MedIncの抽出
x_data = df[['MedInc']].values

#### ダミー変数（定数項）追加

In [ ]:
# ダミー変数（定数項）追加
x = np.insert(x_data, 0, 1.0, axis=1)

#### 目的変数ytの設定

In [ ]:
# 目的変数ytの設定
yt = df['MedianHouseValue'].values

#### 加工結果確認

In [ ]:
# 加工結果確認

print("xとytのshape")
print(f"x.shape: {x.shape}, yt.shape: {yt.shape}")

print('xの先頭5行')
print(x[:5])

print('ytの先頭5要素')
print(yt[:5])

#### 散布図表示

In [ ]:
# 散布図表示
# x(収入)とyt(不動産価格)の関係を散布図表示する

plt.figure(figsize=(6,6))
plt.scatter(x[:,1], yt, s=0.5, c='blue')
plt.title('収入 vs 不動産価格の散布図', fontsize=14)
plt.xlabel('MedInc（収入）', fontsize=13)
plt.ylabel('MedianHouseValue（不動産価格）', fontsize=13)
plt.grid(True)
plt.show()

### 基本関数定義

#### 予測関数

In [ ]:
# 予測関数

def predict(x, w):
    """線形回帰モデルによる予測値を計算"""
    return x @ w

#### 損失関数

In [ ]:
# 損失関数

def compute_loss(yp, yt):
    """平均二乗誤差 (MSE) に基づく損失関数"""
    return np.mean((yp - yt) ** 2) / 2

### 学習

#### 学習関数

In [ ]:
# 学習関数
def train_linear_regression(x, yt, alpha=0.005, iters=5000, his_unit=100):
    """勾配降下法による線形回帰モデルの学習"""
    # M(データ件数)とD(入力データ要素数)の設定
    M, D = x.shape
    # 重みベクトル初期化(全要素1を設定)
    w = np.ones(D)
    # 学習過程記録用
    history = np.zeros((0,2))

    # 繰り返し処理
    for k in range(iters):
        # 予測計算
        yp = predict(x, w)
        # 誤差計算
        yd = yp - yt
        # 勾配(微分)計算
        grad = (x.T @ yd)/M
        # パラメータ修正
        w -=  alpha * grad

        if k % his_unit == 0:
            # 損失計算
            loss = compute_loss(yp, yt)
            # 記録用変数に追記
            history = np.vstack((history, np.array([k, loss])))
            # 結果の画面表示
            print(f"iter={k:5d} | loss={loss:.6f}")
    return w, history

#### 学習

In [ ]:
# 学習

# 学習率と繰り返し回数の設定
alpha = 0.005
iters = 5000
his_unit = 100

# 繰り返し処理
w, history = train_linear_regression(x, yt, alpha=alpha, \
            iters=iters, his_unit=his_unit)

### 結果分析

#### 損失の確認

In [ ]:
# 損失の確認
print(f"損失初期値: {history[0,1]:.06f}")
print(f"損失最終値: {history[-1,1]:.06f}")

#### 学習曲線(損失)

In [ ]:
# 学習曲線(損失)
plt.figure(figsize=(6,6))
plt.plot(history[1:,0], history[1:,1])
plt.title('学習曲線(損失)', fontsize=14)
plt.grid()
plt.xlabel('繰り返し回数', fontsize=13)
plt.ylabel('損失', fontsize=13)
plt.show()

#### 回帰直線の可視化

In [ ]:
# 回帰直線の可視化

# 回帰直線の座標計算
xall = x[:,1].ravel()
xl = np.array([[1, xall.min()], [1, xall.max()]])
yl = predict(xl, w)

# グラフ描画
plt.figure(figsize=(6,6))
plt.scatter(x[:,1], yt, s=0.1, c='b', label='観測データ')
plt.plot(xl[:,1], yl, c='k', lw=2, label='学習後の回帰直線')
plt.title('散布図と回帰直線', fontsize=14)
plt.xlabel('収入', fontsize=14)
plt.ylabel('不動産価格', fontsize=14)
plt.legend()
plt.grid()
plt.show()

### 重回帰

#### データ加工(重回帰)

In [ ]:
# データ加工(重回帰)

# 特徴量AveRoomsの抽出
x_add = df[['AveRooms']].values

# 特徴量AveRoomsの説明変数xへの追加
x2 = np.hstack((x, x_add))

# 結果確認
print("x2のshape")
print(f"x2.shape: {x2.shape}")
print('x2の先頭5行')
print(x2[:5])

#### 学習(重回帰)

In [ ]:
# 学習(重回帰)

# 学習率と繰り返し回数の設定
alpha = 0.005
iters = 5000
his_unit = 100

# 繰り返し処理
w2, history2 = train_linear_regression(x2, yt, alpha=alpha, \
            iters=iters, his_unit=his_unit)

#### 結果分析(重回帰)

In [ ]:
# 結果分析(重回帰)

# 損失の確認
print(f"損失 初期値 : {history2[0,1]:.06f}")
print(f"損失 最終値 : {history2[-1,1]:.06f}")

# 学習曲線(損失　重回帰モデル)
plt.figure(figsize=(6,6))
plt.plot(history[1:,0], history2[1:,1], color='blue')
plt.title('学習曲線（損失　重回帰モデル）', fontsize=14)
plt.xlabel('繰り返し回数', fontsize=13)
plt.ylabel('損失', fontsize=13)
plt.grid(True)
plt.show()

### バージョン確認

In [ ]:
!pip install watermark -qq
%load_ext watermark
%watermark --iversions